# Welcome to the Connecting Python to SQL lab!
In this lab, you will be working with the Sakila database on movie rentals. Specifically, you will be practicing how to do basic SQL queries using Python. By connecting Python to SQL, you can leverage the power of both languages to efficiently manipulate and analyze large datasets. Throughout this lab, you will practice how to use Python to retrieve and manipulate data stored in the Sakila database using SQL queries. Let's get started!

## Challenge
In this lab, the objective is to identify the customers who were active in both May and June, and how did their activity differ between months. To achieve this, follow these steps:

### 1.Establish a connection between Python and the Sakila database.

In [2]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import getpass  # Para obtener la contraseña sin mostrar la entrada

# Obtener la contraseña de forma segura
password = getpass.getpass(prompt='Introduce tu contraseña MySQL: ')

# Configuración de la base de datos y la conexión
bd = "sakila"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd
engine = create_engine(connection_string)


### 2. Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:
engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
month: an integer representing the month for which rental data is to be retrieved.
year: an integer representing the year for which rental data is to be retrieved.

The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [6]:
# Definir la función rentals_month
def rentals_month(engine, month, year):
    consulta = f"""
    SELECT customer_id, rental_id, rental_date
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    df = pd.read_sql(consulta, con=engine)
    return df

### 3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.
The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

Hint: Consider making use of pandas groupby()

In [7]:
# Definir la función rental_count_month
def rental_count_month(df, month, year):
    rental_count = df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    return rental_count

### 4. Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [12]:
def compare_rentals(df1, df2):
    df_combined = pd.merge(df1, df2, on='customer_id', how='inner')
    df_combined['difference'] = df_combined.iloc[:, 1] - df_combined.iloc[:, 2]
    return df_combined

### Final implementation

In [10]:
# Obtener los datos de alquileres para mayo y junio
df_may = rentals_month(engine, 5, 2005)
df_june = rentals_month(engine, 6, 2005)

# Contar alquileres por cliente
df_may_count = rental_count_month(df_may, 5, 2005)
df_june_count = rental_count_month(df_june, 6, 2005)

# Comparar alquileres entre mayo y junio
df_comparison = compare_rentals(df_may_count, df_june_count)

# Mostrar los primeros resultados
df_comparison

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2,7,-5
1,2,1,1,0
2,3,2,4,-2
3,5,3,5,-2
4,6,3,4,-1
...,...,...,...,...
507,594,4,6,-2
508,595,1,2,-1
509,596,6,2,4
510,597,2,3,-1
